# Solutions to Practice Problems based on Week 3 content

## General Course Logistics

* Review the project assignment. Find a group and start brainstorming topics, even if it's a group of size 1!


## Min-Cost Network Flow Concepts and Definitions

* Find a better upper bound on the "Max Flow" example given in Module 5 Lecture 7.

## Answer

There are several answers to this question! One possible upper bound is the sum of the capacities of the arcs entering the sink: 3-5 + 4-5 = 3+14 = 17. We can guarantee that we won't be able to send more than 17 units of flow through the network.

### Max Flow Problem

Last week you figured out how to feed the Martian colonists. This week, your challenge is to provide them with water. There are three bases that will need water: Kepler, Copernicus, and Tycho. Two wells have been installed. There are five pipe intersections where water can be rerouted. The pipes have specified capacities (in gallons/day) as seen in the network below:

![](mars_network.png)

Well 1 can supply up to 30 gallons per day, and Well 2 can supply up to 27 gallons per day. Kepler needs 17 gallons/day, Copernicus needs 15 gallons/day, and Tycho needs 21 gallons/day. What is the maximum flow through the network? Can the daily demand be met?

Find a minimum cut in the network above.

## Answer

This is a modified max flow problem. First, we will solve it as a typical max flow problem then modify as needed based on the solution

In [1]:
using JuMP, Clp

nodes = [:s,:1,:2,:3,:4,:5,:6,:7,:k,:c,:t,:si] # create list of nodes. create a dummy source node and dummy sink node.
# create list of all arcs in the network. include dummy arce from the source to well 1 and well 2, 
# from kepler, copernicus, and tycho to sink, and sink to source
arcs = [(:s,:1), (:s,:2), (:1,:3), (:1, :4), (:1, :5), (:2,:4), (:2,:5), (:3,:5),(:3,:7),
        (:4,:5), (:4,:6),(:5,:k),(:5,:c),(:5,:t),(:6,:c),(:6,:t),(7,:k),(:7,:c),(:k,:si),(:c,:si),(:t,:si), (:si,:s)]

# dictionary of arc capacities
capacity = Dict(zip(arcs,[30 27 21 11 16 22 5 16 9 2 23 14 10 16 10 9 8 10 100 100 100 100]))

m = Model(Clp.Optimizer)

#variables represent flow on each arc
@variable(m, x[arcs] >= 0)

# maximize total flow on arc from sink to source
@objective(m, Min, -x[(:si,:s)])

@constraint(m, cap[a in arcs], x[a] <= capacity[a]) # obey capacity restrictions

# balance flow on every node
@constraint(m, flow[i in nodes], sum(x[a] for a in arcs if a[1] == i) == sum(x[a] for a in arcs if a[2] == i))
 
optimize!(m)
                            
println("Total flow through network: ", -objective_value(m))
                                                        
println("check if demand met: ")
println("flow to kepler (should be at least 17): ", value(x[(:k,:si)]))
println("flow to copernicus (should be at least 15): ", value(x[(:c,:si)]))
println("flow to tycho (should be at least 21): ", value(x[(:t,:si)]))

Total flow through network: 56.0
check if demand met: 
flow to kepler (should be at least 17): 19.0
flow to copernicus (should be at least 15): 20.0
flow to tycho (should be at least 21): 17.0
Coin0506I Presolve 3 (-31) rows, 5 (-17) columns and 10 (-56) elements
Clp0006I 0  Obj -5 Dual inf 1.999998 (2)
Clp0006I 3  Obj -56
Clp0000I Optimal - objective value -56
Coin0511I After Postsolve, objective -56, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -56 - 3 iterations time 0.122, Presolve 0.11


We aren't meeting the Tycho demand, so add a constraint and re-solve:

In [2]:
@constraint(m, x[(:t,:si)] == 21) # >= okay too
optimize!(m)
                            
println("Total flow through network: ", -objective_value(m))
                                                        
println("check if demand met: ")
println("flow to kepler (should be at least 17): ", value(x[(:k,:si)]))
println("flow to copernicus (should be at least 15): ", value(x[(:c,:si)]))
println("flow to tycho (should be at least 21): ", value(x[(:t,:si)]))

Total flow through network: 56.0
check if demand met: 
flow to kepler (should be at least 17): 14.0
flow to copernicus (should be at least 15): 21.0
flow to tycho (should be at least 21): 21.0
Coin0506I Presolve 5 (-30) rows, 12 (-10) columns and 24 (-43) elements
Clp0006I 0  Obj 0 Primal inf 41.999997 (3) Dual inf 2.999997 (3)
Clp0006I 9  Obj -56
Clp0000I Optimal - objective value -56
Coin0511I After Postsolve, objective -56, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -56 - 9 iterations time 0.002, Presolve 0.00


Now we aren't meeting the Kepler demand, so add a constraint and re-solve:

In [3]:
@constraint(m, x[(:k,:si)] == 17) # >= okay too
optimize!(m)
                            
println("Total flow through network: ", -objective_value(m))
                                                        
println("check if demand met: ")
println("flow to kepler (should be at least 17): ", value(x[(:k,:si)]))
println("flow to copernicus (should be at least 15): ", value(x[(:c,:si)]))
println("flow to tycho (should be at least 21): ", value(x[(:t,:si)]))

Total flow through network: 56.0
check if demand met: 
flow to kepler (should be at least 17): 17.0
flow to copernicus (should be at least 15): 18.0
flow to tycho (should be at least 21): 21.0
Coin0506I Presolve 5 (-31) rows, 11 (-11) columns and 22 (-46) elements
Clp0006I 0  Obj -48 Primal inf 52.199997 (4) Dual inf 1.999998 (2)
Clp0006I 9  Obj -56
Clp0000I Optimal - objective value -56
Coin0511I After Postsolve, objective -56, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective -56 - 9 iterations time 0.002, Presolve 0.00


This model (max flow with two extra constraints) gives a feasible flow that meets demand!

Now we need to find a minimum cut. Remmber that max flow = min cut, so we need to find a set of arcs that separate the source from the sink such that the sum of the capacities on the arcs = 56. We can use dual varaiable values to recover a minimum cut:

In [4]:
min_cut = 0
for a in arcs
    # if the dual variable is nonzero, the primal capacity constraint is active
    if abs(dual(cap[a])) > 10e-5
        # print the arc where the associated primal capacity is active
        println("Arc in cut: ", a, " (Capacity: ", capacity[a], ")")
        min_cut = min_cut + capacity[a]
    end
end
println("Total capacity of this minimum cut (should be 56): ", min_cut)

Arc in cut: (:s, 1) (Capacity: 30)
Arc in cut: (2, 5) (Capacity: 5)
Arc in cut: (4, 5) (Capacity: 2)
Arc in cut: (6, :c) (Capacity: 10)
Arc in cut: (6, :t) (Capacity: 9)
Total capacity of this minimum cut (should be 56): 56



## Duality and Sensitivity Analysis

* Take the dual of the dual. Covert the general-form dual to standard form, then take the dual.

## Answer

The main steps are shown in Module 6, Lecture 1. Try this on your own and check your work against the steps in the slides!


* Prove the theorem of Weak Duality for standard form LPs.

## Answer

We can use similar logic as was done with Top Brass. I will only show high level steps:

* We have a primal problem: $$P = \{\max c^Tx | Ax \leq b; x \geq 0 \}$$. Let $x^*$ be the optimal solution, $p^*$ the optimal objective.
* Of course, if $\hat{x}$ is feasible, $c^T\hat{x} \leq c^Tx^* = p^*$. 
* How can we find upper bounds? Take a vector $\lambda \geq 0$. Then certainly $\lambda^T(Ax) \leq \lambda^Tb$.
* Further, if we choose $\lambda$ such that $\lambda^T(Ax) \geq c^Tx$ for any feasible $x$, than we have $\lambda^Tb \geq \lambda^T(Ax) \geq c^Tx$.
* Now we would like to find a $\lambda \geq 0$ that gives us the minimum possible lower bound.
* In other words, we would like to find $\lambda \geq 0$ such that we minimize $b^T\lambda$ and $\lambda^T(Ax) \geq c^Tx$.
* A final obeservation: if we pick $\lambda$ such that $\lambda^TA\geq c$, then because all feasible $x$ are $\geq 0$, certainly we satisfy $x^T(\lambda^TA) \geq x^Tc$.
* Thus, we always have an upper bound on $P$ if we solve the problem $D = \{\min b^T\lambda | A^T\lambda \geq c; \lambda \geq 0\}$.
* Any other choice of $\lambda$ that satisfies the constraints in $D$ will be an upper bound on $d^*$.
* And we are done!


* Pick your favorite example from class so far and take the dual of the problem.

## Answer
Answers will vary!


* Recall the Stigler diet problem from Week 1's practice problems. Suppose you wanted to find an even cheaper solution and you found an online retailer that would sell you calcium supplements and protein powder.  The calcium supplements contain 0.5 g of calcium each and the protein powder comes in packets of 10 g.

     (a)  How much would we be willing to pay for the calcium supplements?  How much for the proteinpowder? Hint: Use duality!

     (b)  Suppose we could buy the calcium supplments for \\$0.01 / pill.  What is your new optimal diet? How much do you save per year, compared to your diet from the original problem (without thesupplements)?

## Answer

We'll use duality to answer hese questions. First, solve the primal:

In [5]:
using DataFrames, CSV, NamedArrays

#Note that the new syntax differs a bit from what is in stigler.jl. This is the non-deprecated version.
df = CSV.read("stigler.csv",DataFrame, delim =',');
# the names of the DataFrame (header) are the nutrients
nutrients = propertynames(df)[2:end]
# create a list of foods from the diet array
foods = convert(Array,df[2:end,1]) # turn dataframe into Array
# create a dictionary of the min requirement of each nutrient
min_daily_req = Dict(zip(nutrients,df[1,2:end]))

# create a NamedArray that specifies how much of each nutrient each food provides
using NamedArrays
food_nutrient_matrix = Matrix(df[2:end,2:end]) # turn dataframe into Matrix
# rows are foods, columns are nutrients
food_nutrient_array = NamedArray(food_nutrient_matrix, (foods, nutrients), ("foods","nutrients"))

using JuMP,  Clp
m = Model(Clp.Optimizer) # create model
set_optimizer_attribute(m,"LogLevel",0)
@variable(m, x[foods] >= 0)

@objective(m, Min, sum(x))

@constraint(m, meet_req[n in nutrients], sum(food_nutrient_array[i,n] * x[i] for i in foods) >= min_daily_req[n])

optimize!(m)

println("Annual cost of this diet \$", 365.25objective_value(m),2)
println("How much of each food should Stigler eat every day? ")
for i in foods
    if value(x[i]) > 10e-5
        println(i, ": ", value(x[i]))
    end
end

# to determine how much we will pay for calcium and protein, we
# get the shadow prices of those constraints (dual variable values)
println("We would pay up to \$ ", dual(meet_req[Symbol("Protein (g)")]), " for a gram of Protein")
println("We would pay up to \$ ", round(dual(meet_req[Symbol("Calcium (g)")]),digits=2), " for a gram of Calcium")

Annual cost of this diet $39.688897115017942
How much of each food should Stigler eat every day? 
Wheat Flour (Enriched): 0.02951906167648827
Liver (Beef): 0.0018925572907052643
Cabbage: 0.011214435246144865
Spinach: 0.005007660466725203
Navy Beans, Dried: 0.061028563526693246
We would pay up to $ 0.0 for a gram of Protein
We would pay up to $ 0.03 for a gram of Calcium


(a) Since a Calcium pill contains 0.5 grams of Calcium, we would only be willing to pay \\$0.015 for a Calcium supplement. We wouldn't pay for protein powder.

(b) We can add a new "food" to our problem that represents \\$1 worth of Calcium supplements (i.e., since 0.5g Calcium Supp. = \\$0.01, 50g Calcium Supp. = \\$1). The easiest way to answer this question is to solve the problem again to see how much Calcium Supplement we add to our diet:

In [6]:

foods2 = vcat(foods, "Calcium Supp")
# i know that Calcium is the 3rd column of the food_nutrient_array
new_row = zeros(length(nutrients))
new_row[3] = 50
food_nutrient_matrix2 = vcat(food_nutrient_matrix,new_row')

food_nutrient_array2 = NamedArray(food_nutrient_matrix2, (foods2, nutrients), ("foods","nutrients"))

# build and solve the new model
m = Model(Clp.Optimizer) 
set_optimizer_attribute(m,"LogLevel",0)
@variable(m, x[foods2] >= 0)

@objective(m, Min, sum(x))
println(min_daily_req)
@constraint(m, meet_req[n in nutrients], sum(food_nutrient_array2[i,n] * x[i] for i in foods2) >= min_daily_req[n])


optimize!(m)

println("Annual cost of this diet \$", 365.25objective_value(m),2)
println("How much of each food should Stigler eat every day? ")
for i in foods2
    if value(x[i]) > 10e-5
        println(i, ": ", value(x[i]))
    end
end

Dict{Symbol, Any}(Symbol("Riboflavin (mg)") => 2.7, Symbol("Calcium (g)") => 0.8, Symbol("Niacin (mg)") => 18, Symbol("Ascorbic Acid (mg)") => 75, Symbol("Thiamine (mg)") => 1.8, Symbol("Vitamin A (1000 IU)") => 5.0, Symbol("Protein (g)") => 70, Symbol("Calories (1000)") => 3.0, Symbol("Iron (mg)") => 12)
Annual cost of this diet $37.02358863983312
How much of each food should Stigler eat every day? 
Wheat Flour (Enriched): 0.06598060307911847
Liver (Beef): 0.00784433892120114
Cabbage: 0.011195027632464827
Spinach: 0.003911295356684479
Calcium Supp: 0.012433796310553268


We save about \\$2 by using 0.01 units of Calcium Supplements ever day.


* Consider the following linear program:

$\begin{align*}
\underset{x_1,x_2}{\max} \ & 3x_1 + x_2& \\
\text{s.t.} \ & 2x_1 + x_2 \leq 4\\
& 3x_1 + 2x_2 \geq 6& \\
 & 4x_1 + 2x_2 = 7\\
& x_1, x_2 \text{free}
\end{align*}$

(a) Find a solution to this linear program.

(b) Using the theorem of Complementary Slackness and the optimal primal solution, find the optimal dual solution.

(c) How would our solution and objective value change if we increased the coefficient on $x_1$ in the objective to 5? What if we change the right-hand side on the 3rd constraint to 8?

## Answer

(a)

In [7]:
# First, just solve the original problem

using Clp, JuMP

m = Model(Clp.Optimizer)
set_optimizer_attribute(m,"LogLevel",0)

@variable(m,x1)
@variable(m,x2)

@objective(m, Max, 3x1 + x2)

@constraint(m, c1, 2x1+x2 <= 4)
@constraint(m, c2, 3x1 + 2x2 >= 6)
@constraint(m, c3, 4x1 + 2x2 == 7)

optimize!(m)
println(m)
println("x1 = ", value(x1))
println("x2 = ", value(x2))
println(dual(c2))
println("objective = ", objective_value(m))

Max 3 x1 + x2
Subject to
 c3 : 4 x1 + 2 x2 == 7.0
 c2 : 3 x1 + 2 x2 >= 6.0
 c1 : 2 x1 + x2 <= 4.0

x1 = 1.0
x2 = 1.5
1.0
objective = 4.5


(b) First we can write the dual problem:

$\begin{align}
\min & 4\lambda_1 + 6\lambda_2 + 7\lambda_3\\
\text{s.t.}& 2\lambda_1 + 3\lambda_2 + 4\lambda_3 = 3\\
& \lambda_1 + 2\lambda_2 + 2\lambda_3 = 1\\
& \lambda_1 \geq 0, \lambda_2 \leq 0, \lambda_3 \text{ free}.
\end{align}$

Because of strong duality, we know the dual objective will also be 4.5. Let $\lambda_i$ be the dual variable associated with constraints $i=1,2,3$ in the primal problem. 

First, we find which dual variables should be 0 by finding constraints with slack. Constraint 1 has 2(1)+1.5 = 2+1.5 < 4. In constraint 2, we have 3(1) + 2(1.5) = 6. Constraint 3 is an equality. Thus, only $\lambda_1 = 0$. We have three equations and 2 unknowns, now, so let's take the objective and the first dual constraint and solve for $\lambda_2$ and $\lambda_3$.

$$6\lambda_2 + 7\lambda_3 = 4.5$$

$$3\lambda_2 + 4\lambda_3 = 3$$

Solving for $\lambda_2$ and $\lambda_3$ leaves us with: $\lambda = (0,-1,1.5)$. We can double-check this gives us the right objective by solving $4(0) + 6(-1) + 7(1.5) = 4.5

(c) Increasing the coefficient on $x_1$ by 2 increases our primal cost by $2 * x_1^* = 2(1) = 2$. Our new objective should be 6.5. 

Increasing the right-hand side of constraint 3 by 1 should increase our dual cost by $(1 * \lambda_3 = 1(1.5) = 1.5). In other words, our new primal objective should be 4.5+1.5 = 6. 

We verify both below:

In [8]:
# change primal cost

m = Model(Clp.Optimizer)
set_optimizer_attribute(m,"LogLevel",0)

@variable(m,x1)
@variable(m,x2)

@objective(m, Max, 5x1 + x2)

@constraint(m, c1, 2x1+x2 <= 4)
@constraint(m, c2, 3x1 + 2x2 >= 6)
@constraint(m, c3, 4x1 + 2x2 == 7)

optimize!(m)

println("objective w/ changed primal cost = ", objective_value(m))
println()
# change primal constraint

m = Model(Clp.Optimizer)
set_optimizer_attribute(m,"LogLevel",0)

@variable(m,x1)
@variable(m,x2)

@objective(m, Max, 3x1 + x2)

@constraint(m, c1, 2x1+x2 <= 4)
@constraint(m, c2, 3x1 + 2x2 >= 6)
@constraint(m, c3, 4x1 + 2x2 == 8)

optimize!(m)

println("objective w/ changed primal constraint = ", objective_value(m))

objective w/ changed primal cost = 6.5

objective w/ changed primal constraint = 6.0


Yay! This matches our predictions.